# Using Postgres SQL

**Disadvantages of SQLite**
SQLite is one of the most common database engines, and has many advantages:

- The database is stored in a single file, making it portable.
- We can use a SQLite database directly from Python, and don't need a separate program running.
- It implements most SQL commands, enabling you to use most of the statements you're familiar with.

However,  SQLite has a few downsides that make other database engines more attractive:

- Only one process at a time can write to the database. 
- We can't take advantage of performance features, such as caching. Because a SQLite database is a single file, and it doesn't require a special program to run, it can't have performance optimizations like caching.
- SQLite doesn't have any built-in security.

**Postgres SQL Overview**
[PostgreSQL](https://www.postgresql.org/), also known as `Postgres`, is an extremely powerful database engine. At a high level, PostgreSQL consists of two pieces, a server and clients. The server is a program that manages databases and handles queries. Clients communicate back and forth to the server. Only the server ever directly accesses the databases -- the clients can only make requests to the server. 

One of the advantages of this model is that multiple clients can communicate with the server at the same time. This allows multiple processes to write to a database at the same time.

It's possible to run a PostgreSQL server either remotely or locally. If it's remote, we connect to it via the internet. If it's local, we connect to it on our own machine. In both cases, we'll be connecting to PostgreSQL via a system port.

By default, PostgreSQL uses `port 5432` to communicate with the outside world. If we start a PostgreSQL server, it will listen for incoming connections on port `5432`. Clients will be able to connect to the server using this port. If we start a client, you'll have to specify which server to connect to, along with the port to connect to.


### Postgres Client Psycopg2
There are many clients for `PostgreSQL`, including graphical clients. The most common Python client for `PostgreSQL` is called [`psycopg2`](https://www.psycopg.org/docs/cursor.html). Connecting to a PostgreSQL database using psycopg2 is similar to connecting to a SQLite database using the sqlite3 library. psycopg2 also uses `Connection` and `Cursor` objects.

We'd connect to a database using psycopg2 like this:
```
import psycopg2
conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()
```
We have to specify both a database name and a user name. A PostgreSQL server can have multiple databases and multiple users, so we need to specify which user we're connecting as, and which database we're connecting to.
When PostgreSQL is first installed, the default user account is called postgres, with an associated database called postgres.
We didn't specify a server to connect to. `Psycopg2` will default to connecting to `port 5432` on the current computer.

When we're done with a Connection object, we should close it to avoid issues where one connection prevents another from executing a query. Closing a connection will terminate the client's connection with the PostgreSQL server. We can close a connection like this:
```
conn.close()
```
**Exercise**
Import the psycopg2 library.
Connect to the dq database with the user dq.
Initialize a Cursor object from the connection.
Use the print function to display the Cursor object.
Close the Connection using the close method

`pip -install psycopg2`

`import psycopg2
conn=psycopg2.connect("dbname=jobs user=js")
cur=conn.cursor()
print(cur)
conn.close()`                      

### Creating a Table

Once we've connected to a database, we can create a table inside that database using `CREATE TABLE` statement:
```
CREATE TABLE tableName(
   column1 dataType1 PRIMARY KEY,
   column2 dataType2,
   column3 dataType3,
   ...
);
```
**Exercise**
- Connect to the jobs database as the user js
- Write a SQL query that creates a table called `notes` in the dq database, with the following columns and data types:
- `id` -- integer data type, and is a primary key.
- `body` -- text data type.
- `title` -- text data type.
- Execute the query using the `execute` method.
- Close the Connection using the `close` method.

```
import psycopg2
conn=psycopg2.connect("dbname=jobs user=js")
cur=conn.cursor()
query=*CREATE TABLE notes (id integer PRIMARY KEY, body text, title text);"
cur.execute(query)

conn.close()
```

### SQL Transactions

- If we now check the database `jobs`, we would notice that there actually isn't a `note` table inside it. This isn't a bug -- it's because of a concept called `SQL transactions`. With SQLite, every query we made that modified the data was immediately executed, and immediately changed the database.
With `PostgreSQL`, we're dealing with multiple users who could be changing the database at the same time.

Let's imagine a simple scenario where we're keeping track of accounts for different customers of a bank. We could write a simple query to create a table for this:
```
CREATE TABLE accounts(
   id integer PRIMARY KEY,
   name text,
   balance float
);
```

Let's say we have the following two rows in the table:
```
id    name    balance
1     Jim     100
2     Sue     200
```
Let's say Sue gives 100 dollars to Jim. We could model this with two queries:
```
UPDATE accounts SET balance=200 WHERE name="Jim";

UPDATE accounts SET balance=100 WHERE name="Sue";
```
n the above example, we remove 100 dollars from Sue, and add 100 dollars to Jim. Let's say either the second UPDATE statement has an error in it, the database fails, or another user has a conflicting query. The first query would run properly, but the second would fail. That would result in the following:
```
id    name    balance
1     Jim     200
2     Sue     200
```
Jim would be credited 100 dollars, but 100 dollars would not be removed from Sue. This would cause the bank to lose money.

Transactions prevent this type of behavior by ensuring that all the queries in a transaction block are executed at the same time. If any of the transactions fail, the whole group fails, and no changes are made to the database at all.

Whenever we open a Connection in psycopg2, a new transaction will automatically be created. All queries run up until the `commit` method is called will be placed into the same transaction block. When commit is called, the PostgreSQL engine will run all the queries at once.

If we don't want to apply the changes in the transaction block, we can call the `rollback` method to remove the transaction. Not calling either `commit` or `rollback` will cause the transaction to stay in a pending state, and will result in the changes not being applied to the database.

**Exercise**
- Connect to the jobs database as the user js.
- Write a SQL query that creates a table called notes in the dq database, with the following columns and data types:
- `id` -- integer data type, and is a primary key.
- `body` -- text data type.
- `title` -- text data type.
- Execute the query using the `execute` method.
Use the `commit` method on the Connection object to apply the changes in the transaction to the database.
Close the Connection.

```
import psycopg2
conn=psycopg2.connect("dbname=jobs user=js")
cur=conn.cursor()
query="CREATE TABLE notes (
id integer PRIMARY KEY,
body text,
title text);"
cur.execute(query)
conn.commit()
conn.close()
```

### Auto Committing
There are cases when you won't want to manage a transaction, and you'll instead want changes right away.

Some changes also have such widespread effects that they can't be wrapped inside of a transaction. One example of this is creating a database. When creating a database, we'll need to activate `autocommit` mode first. To activate `autocommit` mode, we'll need to set the `autocommit` property of the Connection object to `True`

```
conn = psycopg2.connect("dbname=jobs user=js")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE TABLE notes(id integer PRIMARY KEY, body text, title text)")
```
Above command will create a table called `notes` without having to explicitly `commit` the transaction.

**Exercise**

- Connect to the jobs database as the user js.
- Set the `autocommit` property of the Connection object to `True`.
- Write a SQL query that creates a table called `facts` in the dq database, with the following columns and data types:
- `id` -- integer data type, and is a primary key.
- `country` -- text data type.
- `value` -- text data type.
- Execute the query using the `execute` method.
- Close the Connection.


```
import psycopg2
conn=psycopg2.connect("dbname=dq user=dq")
conn.autocommit=True
cur=conn.cursor()
query="CREATE TABLE facts (id integer PRIMARY KEY, country text, value text);"
cur.execute(query)
conn.close()
```

### Executing Queries

We can issue `SELECT` queries against a database using the `execute` method, along with the `fetchall` and `fetchone` methods to retrieve results:

```
cur.execute("SELECT * FROM notes;")
rows = cur.fetchall()
print(rows)
```
We can use fol to insert rows in tables (for all columns):
```
INSERT INTO tableName
VALUES (value1, value2, ...);
```
```
INSERT INTO notes
VALUES (1, 'This is my note text.', 'Test note');
```
**Exercise**
- Execute a SQL query that inserts a row into the notes table with the following values:
- `id` -- 1
- `body` -- 'Do more jogging.'
- `title` -- 'Exercise reminder'.
- Execute a SQL query that selects all of the rows from the notes table.
- Fetch all of the results and print them out.
- Commit your changes and close the Connection.



```
import psycopg2
conn=psycopg2.connect("dbname=dq user=dq")
cur=conn.cursor()
query = "INSERT INTO notes VALUES (1,'Do more jogging','Exercise reminder');"
cur.execute(query)
conn.commit()
cur.execute("SELECT * from notes;")
rows=cur.fetchall()
print(rows)
```

### Creating Databases
Postgres SQL allows us to create multiple databases. Different databases are generally used to hold information about different applications.
We can create a database using the CREATE DATABASE SQL statement:
`CREATE DATABASE dbName;`
We can specify the user who will own the database when we create it as well, using the `OWNER` statement: `CREATE DATABASE notes OWNER postgres;`

The owner of a database is the only one that can access and modify a database, unless they give permission to other users. An exception is superusers, who can perform any action on any database without being given permission.

**Exercise**
- Connect to the jobs database with the user js.
- Set the connection to `autocommit` mode.
- Create a database called `income` where the owner is the user js.
- Close the Connection.



```
conn = psycopg2.connect("dbname=jobs user=js")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE income OWNER js")
conn.close()
```

### Deleting Databases
We can delete a database using the `DROP DATABASE` statement. The `DROP DATABASE` statement will immediately remove a database, provided the user executing the query has the right permissions. It should be used with caution when working with real data.
`DROP DATABASE dbName;`

**Exercise**
- Connect to the jobs database with the user js.
- Set the connection to `autocommit` mode.
- Drop the income database.
- Close the Connection.
